In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm 

In [ ]:
import pickle

In [ ]:
filepath = 'C:/Users/g20/OneDrive/바탕 화면/chromedriver_win32/miraeasset.csv'
with open(filepath,'rb') as lf:
    data = pickle.load(lf)

In [ ]:
# 한글이 편할 경우  
data = data.rename(columns = {
'ADD_WITHOUT_CASH_OUT': '현금의유출이없는비용등가산',
'ASSET_SUM': 			'자산총계',
'BASE_YM': 				'회계년월',
'BHJS_CNT': 				'발행주식수', 
'BIZ_ACT_CASH': 			'영업활동현금', 
'BIZ_CTAX_COST': 			'계속사업법인세비용',
'BIZ_PROFIT': 			'영업이익', 
'BIZ_PROFIT_BEFORE_TAX':  '세전계속사업이익 ',
'CAPITAL_AMT': 			'자본금 ',
'CAPITAL_SUM': 			'자본총계',
'CAPITAL_SURPLUS_AMT': 	'자본잉여금',
'CASH_ASSET':				'현금및현금성자산', 
'CASH_GROW': 				'현금의증가', 
'CHANGE_FROM_BIZ_ACT': 	'영업활동으로인한자산및부채의변동',
'CO_NM': 					'기업명칭'              , 
'CODE': 					'기업코드'              ,
'COMMON_CAPITAL': 		'보통주자본금'            ,
'CTAX_EFFECT': 			'법인세효과'             ,
'DATA_TP_CODE': 			'데이터구분'             ,
'DEBT_SUM': 				'부채총계'   ,         
'ZAMU_CASH_FLOW':'재무현금흐름', 
'DOMIN_STOCK_PROFIT'	:	'지배주주순이익',
'END_TERM_CASH'			:	'기말현금',
'ETL_PROC_TIME '		:	'ETL처리시간',
'EXP_RATE'				:	'수출비중',
'FIX_ASSET'				:	'고정자산',
'FIX_DEBT'				:	'고정부채',
'FLOW_ASSET'			:	'유동자산',
'FLOW_DEBT'				:	'유동부채',
'IFRS_TP_CODE'			:	'개별연결구분',
'INTEREST_COST'			:	'이자비용',
'INTEREST_PROFIT'		:	'이자수익',
'INVEST_CASH_FLOW'		:	'투자현금흐름',
'MINUS_WITHOUT_CASH_IN' :	'현금의유입이없는수익등의차감',
'NON_BIZ_COST'			:   '영업외비용',
'NON_BIZ_PROFIT'		:	'영업외수익',
'PART_EVAL_PROFIT'		:	'지분법평가이익',
'PART_EVAL_PROFIT_LOSS'	:	'지분법평가손익',
'PROFIT_BEFORE_CTAX'	:	'법인세비용차감전순이익',
'PROFIT_SURPLUS_AMT'	:	'이익잉여금',
'SALE_AMT'				:	'매출액',
'SALE_COST'				:	'매출원가',
'SALE_COST_RATE'		:	'매출원가율',
'SALE_MANG_AMT'			:	'판매비와관리비',
'SET_TP_CODE'			:	'결산구분',
'START_TERM_CASH'		:	'기초현금',
'STOP_BIZ_PROFIT'		:	'중단사업이익',
'THIS_TERM_PROFIT'		:	'당기순이익_순손실',
'ZAMU_CASH_FLOW	'		:	'재무현금흐름'
})

In [ ]:
def various_index(df,indicators):
    indicators = ['부채총계','자산총계','자본총계','영업이익','당기순이익_순손실','매출액']
    df = df.copy()
    for i in indicators:
        df[i] = df[i].replace(0,np.nan) #각 indicator에서 사용되는 열의 값이 0인경우 nan으로 처리 
    # 부채비율
    df['부채비율'] = df['자본총계'] / df['자산총계']

    # 영업이익 증가율 = (당기 영업이익 - 전기 영업이익) / 전기 영업이익
    df.sort_values(by = ['기업명칭','회계년월'], inplace = True)
    df['영업이익증가율'] = df['영업이익'].pct_change(periods = 1)

    # 매출액증가율 = (당기 매출액 - 전기 매출액) / 전기 매출액
    df['매출액증가율'] = df['매출액'].pct_change(periods = 1)

    # 당기 순이익증가율 = (당기순이익 - 전기의 당기순이익) / 전기의 당기순이익
    df['당기순이익증가율'] = df['당기순이익_순손실'].pct_change(periods = 1)

    # ROA = (당기순이익 / 자산총계)
    df['ROA'] = df['당기순이익_순손실'] / df['자산총계']

    # ROE = (당기순이익 / 평균 자기자본)
    average_equity = df['자본총계'].rolling(2).mean()
    df['ROE'] = df['당기순이익_순손실'] / average_equity

    # ROI = (당기순이익 / 총자본)
    df['ROI'] = df['당기순이익_순손실'] / df['자본총계']

    return df

In [ ]:
# 코스피 종목명 추출
def kospi_total_name():
    kospi_total_name = []
    for val in tqdm(range(1,38)):
        URL = 'https://finance.naver.com/sise/sise_market_sum.nhn?&page={}'.format(val)
        resp = requests.get(URL)
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        kospi_chart_html = soup.select('div.box_type_l tbody')[0]
        for item in kospi_chart_html.find_all('a',class_ = 'tltle'):
            kospi_name = item.get_text()
            kospi_total_name.append(kospi_name)
    return kospi_total_name

In [ ]:
# 코스피에 대한 data 정제
kospi_name = kospi_total_name()

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:11<00:00,  3.15it/s]


In [ ]:
code_name = data.loc[data['기업명칭'].isin(kospi_name)]['기업명칭'].unique().tolist() #코스피 교집합 미래에셋 data 

In [ ]:
len(code_name)

793

In [ ]:
#input:[미래에셋 excel파일의 df,원하는 분기list]
def miraeasset_data(data,date,code_name):
    onereport = []
    report = []
    non_data = []
    total_non_date = []

    for i in tqdm(code_name):
        onecompany_info = data.loc[data['기업명칭'] == i] #한 기업의 모든 정보 가져옴 in 미래에셋

        non_date = []
        for j in date:

            if len(onecompany_info.loc[onecompany_info['회계년월'] == j]) == 0 : # 원하는 date가 없는 기업에 데이터 처리[0]으로 값처리
                non_date.append(j)
                name = data.columns.values.tolist() #col명 배정
                com_code = data[data['기업명칭'] == i ]['기업코드'].unique().tolist() #기업코드
                #기업코드,회계년월,기업명칭, [매출원가율~이익잉여금...:nan]으로 대입 

                if len(com_code)>1: #기업코드명이 바뀌었을 경우 가장 최근 기업코드명으로 지정(ex'F&F')
                    com_code = [com_code[-1]]
                    pass
                onereport.append(pd.DataFrame([com_code+[j,i]+[np.nan]*(len(name)-3)],columns = name)) #없는 경우 nan으로 대입


            else :
                tmp = onecompany_info.loc[onecompany_info['회계년월'] == j] # ex 삼성전자 - [202012, 202112]의 미래에셋 df 가져옴 

                # 하나의 회계년월에 두개의 df를 갖는 주식에 대해 각 열에 대해 가장 많은 값을 가진 row만 남김(ex 한국주철관)
                if len(tmp) >= 2:
                    tmp = pd.DataFrame(tmp.loc[tmp.iloc[:,3:].sum(axis = 1).idxmax()]).T

                onereport.append(tmp)

        # 원하는 회계년월이 없는 기업명과 date값 append
        if len(non_date) != 0:
            non_data.append(i)
            non_data.append(non_date)
    total_report = pd.concat(onereport).reset_index().iloc[:,1:]
    return total_report, non_data

In [ ]:
#분기 date로 list를 만들어 해당 분기에 맞는 주식 데이터프레임 추출함
date=[202012,202112,202203]
total_report, non_data = miraeasset_data(data,date,code_name)

100%|████████████████████████████████████████████████████████████████████████████████| 793/793 [00:14<00:00, 54.28it/s]


In [ ]:
total_report # 제대로 미래에셋 교집합 코스피에 관한 793개 데이터값이 추출됨. 3개년이므로 793*3 = 2379

,기업코드,회계년월,기업명칭,매출원가율,수출비중,발행주식수,자산총계,유동자산,고정자산,현금및현금성자산,...,현금의유입이없는수익등의차감,영업활동으로인한자산및부채의변동,투자현금흐름,재무현금흐름,현금의증가,기초현금,기말현금,계속사업법인세비용,자본잉여금,이익잉여금
0,A095570,202012,AJ네트웍스,0.0,0.0,46822295.0,1204137954690,196445137370,1007692817320,13201292000,...,13379603000,-42724665000,-34830298610,-32175118930,-68461411440,81662703910,13201292470,877548950,103725389530,191523291060
1,A095570,202112,AJ네트웍스,0.0,1.22,46822295.0,1198209311650,130457258750,1067752052900,15573060000,...,-3188307000,-44057399000,-7497732660,11036856810,711522210,14861538170,15573060370,4872496100,103717305900,256256464560
2,A095570,202203,AJ네트웍스,0.0,1.02,46822295.0,1216333773590,157283046090,1059050727490,34542606680,...,3752762000,-51438734000,23924179580,-3132093700,18969546300,15573060370,34542606680,2329927950,103717305900,255080919910
3,A006840,202012,AK홀딩스,0.0,0.0,13247561.0,674491546270,11433465230,663058081040,6597902000,...,108311000,-831160000,-38400000,7828138300,4416696440,2181205870,6597902310,256680740,235647076680,220322984650
4,A006840,202112,AK홀딩스,0.0,0.0,13247561.0,769013515280,4424324710,764589190570,0,...,1162591000,2460659000,-108349308050,26519011370,-82371978320,82371978310,0,-4198870,235647076680,240867392770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,A000540,202112,흥국화재,10.26,0.0,65164245.0,13762260000000,0,0,90288000000,...,88645000000,-78797000000,-73334000000,-4439000000,8289000000,81986000000,90275000000,4020000000,0,268997000000
2375,A000540,202203,흥국화재,11.93,0.0,65164245.0,13944633000000,0,0,90084000000,...,133054000000,-149131000000,-93272000000,18212000000,-2769000000,90275000000,87506000000,12776000000,0,317599000000
2376,A003280,202012,흥아해운,90.59,0.0,26121481.0,284767346550,51275585950,233491760600,3326905000,...,-7001624000,3435267000,4945163600,-16313437630,-7061293080,10388197940,3326904860,777807540,52719543000,-84003911320
2377,A003280,202112,흥아해운,86.24,0.0,240424899.0,247989912180,53674943320,194314968860,36502868000,...,-5440200000,-5259099000,-773264840,3047296470,-397312830,36900181000,36502868170,63454020,96600061000,-68458487160


In [ ]:
total_report[total_report['기업명칭'] == 'DL이앤씨']

,기업코드,회계년월,기업명칭,매출원가율,수출비중,발행주식수,자산총계,유동자산,고정자산,현금및현금성자산,...,현금의유입이없는수익등의차감,영업활동으로인한자산및부채의변동,투자현금흐름,재무현금흐름,현금의증가,기초현금,기말현금,계속사업법인세비용,자본잉여금,이익잉여금
60,A375500,202012,DL이앤씨,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,A375500,202112,DL이앤씨,77.93,0.0,40807508.0,6870763336000,3474170215000,3396593121000,1356003653000,...,30052000000,3767049000,-48308966000,-11372516000,217191300000,1138812352000,1356003653000,65684407000,3271623795000,466459984000
62,A375500,202203,DL이앤씨,82.63,0.0,40807508.0,6872111470000,3491021314000,3381090156000,1339729402000,...,74751832000,-85845880000,-59272301000,-10775530000,-16274250000,1356003653000,1339729402000,38314687000,3269953263000,511609947000


In [ ]:
# 어떤 기업명에 어떤 분기값이 없는지 확인 코드 
for i in range(int(len(non_data)/2)):
    print('{}:{}없음'.format(non_data[2*i],non_data[2*i+1])) #없는 값은 0처리됨. 이때, 바다로19호, 케이탑리츠, 현대약품의 경우 확인필요
    #현대약품은 난관2에 언급했듯 2,5,8,11에 재무제표값만 존재 

DL이앤씨:[202012]없음
F&F:[202012]없음
HD현대:[202012]없음
HJ중공업:[202012]없음
HLB글로벌:[202012]없음
KG스틸:[202012]없음
LX홀딩스:[202012]없음
POSCO홀딩스:[202012]없음
SK스퀘어:[202012]없음
SNT에너지:[202012]없음
TKG휴켐스:[202012]없음
두산에너빌리티:[202012]없음
바다로19호:[202012, 202112, 202203]없음
보령:[202012]없음
에스엠벡셀:[202012]없음
엔에이치스팩19호:[202012]없음
초록뱀헬스케어:[202012]없음
케이탑리츠:[202012, 202112, 202203]없음
콘텐트리중앙:[202012]없음
포스코스틸리온:[202012]없음
현대약품:[202012, 202112, 202203]없음
화승알앤에이:[202012]없음


### 난관 1
3개인 데이터값이 존재,, ->어떤걸 선택할까?
해결) 각 col에 해당하는 값이 많은 row를 선택 

In [ ]:
c = 0 
for i in code_name[600:800]:
    count = sum(total_report['기업명칭'].isin([i]))
    if count != 2:
        print(i,count)

한국주철관 3


In [ ]:
display(total_report.iloc[total_report[total_report['기업명칭'] =='한국주철관'][:2].iloc[:,3:].sum(axis = 1).idxmax()])

기업코드                     A000970
회계년월                      202012
기업명칭                       한국주철관
매출원가율                      81.32
수출비중                         0.0
발행주식수                 22800500.0
자산총계                255079758190
유동자산                100143225960
고정자산                154936532230
현금및현금성자산             36852135000
부채총계                 36903627760
유동부채                 35284781510
고정부채                  1618846250
자본총계                218176130420
자본금                  12000000000
보통주자본금               12000000000
매출액                  32013166640
판매비와관리비               2447001210
매출원가                 26033322260
영업이익                  3532842430
영업외수익                 1177854000
이자수익                    39131000
지분법평가이익                        0
영업외비용                 4201968000
이자비용                    -7369000
지분법평가손익                        0
법인세비용차감전순이익            508725640
세전계속사업이익               508725640
중단사업이익                         0
법인세효과                          0
당기순이익_순손실 

### 난관2 현대약품의 경우 2,5,8,11에 해당하는 값을 산출

In [ ]:
data[data['기업명칭'] == '현대약품'] #현대 약품의 경우 2월,5월,8월,11월을 산출함. 

,기업코드,회계년월,기업명칭,매출원가율,수출비중,발행주식수,자산총계,유동자산,고정자산,현금및현금성자산,...,현금의유입이없는수익등의차감,영업활동으로인한자산및부채의변동,투자현금흐름,ZAMU_CASH_FLOW,현금의증가,기초현금,기말현금,계속사업법인세비용,자본잉여금,이익잉여금
162342,A004310,200005,현대약품,0.00,0.00,28915339.47,68077435000,49682827000,18394608000,9481676000,...,0,0,0,0,0,0,0,0,3858964000,25706545000
162343,A004310,200011,현대약품,0.00,0.00,28915339.47,66672856000,49998738000,16674118000,10940571000,...,0,0,0,0,0,0,0,0,3858964000,28346546000
162344,A004310,200102,현대약품,59.43,0.00,28915339.47,90029134000,56394439000,33634696000,14605858000,...,0,0,0,0,0,0,0,565207000,20819167000,29844443000
162345,A004310,200105,현대약품,47.66,0.06,28915339.47,89955739000,56564489000,33391250000,11688479000,...,0,0,0,0,0,0,0,1317377000,20932927000,32045058000
162346,A004310,200108,현대약품,52.73,0.21,28915339.47,96882225000,62965690000,33916534000,16091608000,...,0,0,0,0,0,0,0,1326900000,20932927000,34948700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162424,A004310,202102,현대약품,60.75,0.00,32000000.00,156450250810,78015382070,78434868750,10178607260,...,450544380,-813458890,-2346588790,-148499750,-3093744000,13272351260,10178607260,-66806480,37720656920,50117723290
162425,A004310,202105,현대약품,58.31,0.00,32000000.00,174734624360,95008972930,79725651420,29392855930,...,-169193840,-905594510,5640443660,11278950530,19214248670,10178607260,29392855930,43417660,37720656920,50544079550
162426,A004310,202108,현대약품,58.92,0.00,32000000.00,174601134940,96426947850,78174187090,22223590770,...,267800570,-1001111720,-9301711380,2266213560,-7169265160,29392855930,22223590770,-199376240,37720656920,50310943210
162427,A004310,202111,현대약품,61.26,0.19,32000000.00,163857416760,84895717680,78961699080,21628096000,...,899633790,2119981940,5980108760,-10145380680,-595494770,22223590770,21628096000,-624778160,37720656920,48354649780


In [ ]:
data[data['기업명칭'] == '바다로19호'] #바다로19호의 경우 상장폐지된 듯 함. 

,기업코드,회계년월,기업명칭,매출원가율,수출비중,발행주식수,자산총계,유동자산,고정자산,현금및현금성자산,...,현금의유입이없는수익등의차감,영업활동으로인한자산및부채의변동,투자현금흐름,ZAMU_CASH_FLOW,현금의증가,기초현금,기말현금,계속사업법인세비용,자본잉여금,이익잉여금
53994,A155900,201112,바다로19호,0.0,0.0,4.0,42190,20000,22190,20000,...,0,0,0,0,0,0,0,0,0,-6503620
53995,A155900,201203,바다로19호,0.0,0.0,4.0,0,0,0,0,...,202314000,0,-43000000000,43001122000,23000,20000,43000,0,0,0
53996,A155900,201206,바다로19호,0.0,0.0,15200004.0,78679922000,768926000,77910995000,279694000,...,180525000,-199224000,-33000000000,32566455000,279651000,43000,279694000,0,2731050000,-2113252000
53997,A155900,201209,바다로19호,0.0,0.0,15200004.0,76795827000,795805000,76000022000,466837000,...,-382839000,199224000,0,-1149346000,164631000,279694000,444325000,0,2731050000,-2644295000
53998,A155900,201212,바다로19호,0.0,0.0,15200004.0,73103422400,776789810,72326632590,435461420,...,332000,192122000,0,-3735768820,-8863580,444325000,435461420,0,138682000,-917942850
53999,A155900,201303,바다로19호,0.0,0.0,15200004.0,75562175000,466936000,75095239000,202248000,...,837843000,0,0,-1124381000,-233214000,435461000,202248000,0,138682000,-313325000
54000,A155900,201306,바다로19호,0.0,0.0,15200004.0,77980417000,346217000,77634200000,95393000,...,147218000,-400861000,0,-1149368000,-106854000,202248000,95393000,0,138682000,140189000
54001,A155900,201309,바다로19호,0.0,0.0,15200004.0,73760854000,1063069000,72697785000,1017492000,...,-985061000,400861000,0,-949477000,922099000,95393000,1017492000,0,138682000,-815264000
54002,A155900,201312,바다로19호,0.0,0.0,15200004.0,73625340560,920269800,72705070760,853409050,...,467000,157519000,0,-1340930590,-164083370,1017492000,853409050,0,138682000,-1123508070
54003,A155900,201403,바다로19호,0.0,0.0,15200004.0,73847737000,902706000,72945032000,832009000,...,171272000,0,0,-1340930000,-21400000,853409000,832009000,0,138682000,-944487000


In [ ]:
# [원하는 주식의 분기별 df, 몇분기 몇년치:num,위의 date의 len과 같음]
def cal_indic(df,num):
    for i in tqdm(range(0,len(df),num)): # ex 1분기*2년: num = 2// 2분기*3년:num = 6
        temp = df.iloc[i:i+num,:] # ex 1분기*2년: num = 2// 2분기*3년:num = 6
        temp = various_index(temp,indicators)
        if i == 0:
            report_total = temp
        else:
            report_total = pd.concat([report_total,temp])
        
    return report_total 

In [ ]:
indicators = ['부채총계','자산총계','자본총계','영업이익','당기순이익_순손실','매출액']

In [ ]:
# 기업별로 묶어서 계산 지금은 기업당 2개씩 행(1분기*2년 = 2)이 있으므로 2로함. 4분기일경우 4분기*년수
cal_indic(total_report,3)

100%|████████████████████████████████████████████████████████████████████████████████| 793/793 [00:10<00:00, 78.55it/s]


,기업코드,회계년월,기업명칭,매출원가율,수출비중,발행주식수,자산총계,유동자산,고정자산,현금및현금성자산,...,계속사업법인세비용,자본잉여금,이익잉여금,부채비율,영업이익증가율,매출액증가율,당기순이익증가율,ROA,ROE,ROI
0,A095570,202012,AJ네트웍스,0.0,0.0,46822295.0,1204137954690,196445137370,1007692817320,13201292000,...,877548950,103725389530,191523291060,0.247849,NaN,NaN,NaN,0.003755,NaN,0.015149
1,A095570,202112,AJ네트웍스,0.0,1.22,46822295.0,1198209311650,130457258750,1067752052900,15573060000,...,4872496100,103717305900,256256464560,0.311121,0.289919,-0.021749,-2.099534,-0.004149,-0.014812,-0.013335
2,A095570,202203,AJ네트웍스,0.0,1.02,46822295.0,1216333773590,157283046090,1059050727490,34542606680,...,2329927950,103717305900,255080919910,0.308821,0.050257,0.297009,-3.194977,0.008971,0.02916,0.029049
3,A006840,202012,AK홀딩스,0.0,0.0,13247561.0,674491546270,11433465230,663058081040,6597902000,...,256680740,235647076680,220322984650,0.766074,NaN,NaN,NaN,-0.032446,NaN,-0.042353
4,A006840,202112,AK홀딩스,0.0,0.0,13247561.0,769013515280,4424324710,764589190570,0,...,-4198870,235647076680,240867392770,0.698693,-0.050029,0.133397,-0.511913,-0.01389,-0.020268,-0.01988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2374,A000540,202112,흥국화재,10.26,0.0,65164245.0,13762260000000,0,0,90288000000,...,4020000000,0,268997000000,0.050184,-0.475644,0.024756,-0.729842,0.00016,0.003122,0.003197
2375,A000540,202203,흥국화재,11.93,0.0,65164245.0,13944633000000,0,0,90084000000,...,12776000000,0,317599000000,0.046968,10.545771,0.039209,21.011775,0.003485,0.072238,0.074206
2376,A003280,202012,흥아해운,90.59,0.0,26121481.0,284767346550,51275585950,233491760600,3326905000,...,777807540,52719543000,-84003911320,-0.070799,NaN,NaN,NaN,-0.111275,NaN,1.571714
2377,A003280,202112,흥아해운,86.24,0.0,240424899.0,247989912180,53674943320,194314968860,36502868000,...,63454020,96600061000,-68458487160,0.436094,-10.021838,0.319709,-0.289931,-0.090731,-0.511455,-0.208054
